<a href="https://colab.research.google.com/github/pranay8297/Stable-Diffusion-Experiments/blob/main/SGD_from_scratch.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install ipdb

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 8.0 MB/s eta 0:00:00


In [2]:
import torch
from torch import nn
from ipdb import set_trace as st

In [100]:
data = torch.randn(10, 4)

In [3]:
# y = 4*x1 + 5*x2 + 2

In [101]:
y = data[:, 0] * 4 + data[:, 1]*5 + data[:, 2]*2 + data[:, 3]*1.2 + 2

In [102]:
data, y

(tensor([[ 0.3368,  1.2659,  0.7874,  0.7350],
         [ 0.1184, -1.5707, -0.6471, -0.7181],
         [-0.7149, -0.3891, -0.3928, -1.3150],
         [-0.6560, -1.9872, -0.0832, -2.4100],
         [ 0.9703,  0.7359,  1.3449, -0.8503],
         [ 0.5559, -0.3309, -1.2239, -1.1382],
         [-0.9164, -0.4503, -0.9206, -1.8818],
         [ 0.5690, -0.9545,  0.5321, -0.5809],
         [ 0.0757,  1.5074, -2.1634, -0.1104],
         [ 0.0139,  1.0272, -0.9465,  1.3147]]),
 tensor([ 12.1337,  -7.5357,  -5.1687, -13.6180,  11.2300,  -1.2443,  -8.0166,
          -0.1293,   5.3803,   6.8759]))

In [103]:
# mean squared error
a = torch.randn(10)
b = torch.randn(10)
((a - b)**2).mean()

tensor(2.6847)

In [42]:
def loss_fn(y, yhat):

  return ((y-yhat.squeeze())**2).mean()

In [104]:
orig_y = y.clone()

In [105]:
x = data.clone()

In [38]:
torch.tensor(np.linspace(-1, 1, 10))

tensor([-1.0000, -0.7778, -0.5556, -0.3333, -0.1111,  0.1111,  0.3333,  0.5556,
         0.7778,  1.0000], dtype=torch.float64)

In [106]:
y = y +  torch.tensor(np.linspace(-1, 1, 10)) * torch.std(y)/3

In [107]:
y = y.to(torch.float32)

In [44]:
from tqdm import tqdm

In [121]:
lr = 0.01
layer = nn.Linear(4, 1, bias = True)
for i in tqdm(range(500)):
  yhat = layer(x)
  loss = loss_fn(y, yhat)
  loss.backward()
  if ((i + 1) % 50) == 0:
    print(f"Loss for epoch: {i+1} is {loss.item()}")
  with torch.no_grad():
    for param in layer.parameters():
      param.sub_(lr * param.grad)
    layer.zero_grad()


100%|██████████| 500/500 [00:00<00:00, 3264.28it/s]

Loss for epoch: 50 is 8.299715042114258
Loss for epoch: 100 is 3.8520901203155518
Loss for epoch: 150 is 2.971248149871826
Loss for epoch: 200 is 2.708824634552002
Loss for epoch: 250 is 2.5910282135009766
Loss for epoch: 300 is 2.520672559738159
Loss for epoch: 350 is 2.4731101989746094
Loss for epoch: 400 is 2.439601421356201
Loss for epoch: 450 is 2.415698766708374
Loss for epoch: 500 is 2.3985819816589355


In [122]:
layer.weight, layer.bias

(Parameter containing:
 tensor([[4.2115, 5.2574, 1.1392, 1.3909]], requires_grad=True),
 Parameter containing:
 tensor([1.8703], requires_grad=True))

In [124]:
y, yhat

(tensor([  9.2270,  -9.7965,  -6.7836, -14.5869,  10.9070,  -0.9213,  -7.0477,
           1.4855,   7.6411,   9.7826]),
 tensor([[ 11.8645],
         [ -7.6248],
         [ -5.4620],
         [-14.7861],
         [ 10.1743],
         [ -0.5067],
         [ -8.0223],
         [ -0.9535],
         [  7.4952],
         [  8.0802]], grad_fn=<AddmmBackward0>))

In [47]:
import matplotlib.pyplot as plt

In [111]:
class MSELoss:
  def forward(self, targ, inp):
    self.targ = targ
    self.inp = inp
    self.out = torch.mean((targ - inp.squeeze())**2)
    return self.out

  def __call__(self, y, yhat):
    return self.forward(y, yhat)

  def backward(self):
    ## Calculate dL/dyhat
    grads = (-2/self.inp.shape[0])*(self.targ - self.inp.squeeze())
    self.inp.grad = grads.unsqueeze(-1)


class Linear:
  def __init__(self, n_in, n_out):
    self.w = torch.randn(n_in, n_out)
    self.b = torch.randn(n_out)

  def forward(self, x):
    self.inp = x
    # bs*n_in, n_in*n_out = bs * n_out
    self.out = torch.matmul(x, self.w) + self.b
    return self.out

  def __call__(self, x):
    return self.forward(x)

  def backward(self):

    self.w.grad = self.inp.t() @ self.out.grad
    self.b.grad = self.out.grad.sum(0)
    self.inp.grad = self.out.grad @ self.w.t()

  def zero_grad(self):
    self.w.grad = torch.zeros_like(self.w)
    self.b.grad = torch.zeros_like(self.b)



In [112]:
class Relu:
  def forward(self, x):
    self.inp = x
    self.out = torch.clamp(x, torch.tensor(0))
    return self.out

  def __call__(self, x):
    return self.forward(x)

  def backward(self):
    self.inp.grad = (self.inp > 0.0).float() * self.out.grad


In [113]:
class Model:

  def __init__(self, layers):
    self.layers = layers

  def forward(self, inp):
    for l in self.layers: inp = l(inp)
    return inp

  def __call__(self, inp):
    return self.forward(inp)

  def backward(self):
    for l in reversed(self.layers): l.backward


In [115]:
loss_fn = MSELoss()
layer = Linear(4, 1)
yhat = layer(x)
loss = loss_fn(y, yhat)

In [116]:
y, yhat.dtype

(tensor([  9.2270,  -9.7965,  -6.7836, -14.5869,  10.9070,  -0.9213,  -7.0477,
           1.4855,   7.6411,   9.7826]),
 torch.float32)

In [118]:
lr = 0.1
layer = Linear(4, 1)
for i in tqdm(range(500)):
  yhat = layer(x)
  loss = loss_fn(y, yhat)
  loss_fn.backward()
  layer.backward()
  if (i + 1)%50 == 0:
    print(f"\n Loss for epoch: {i+1} is {loss.item()}")
  with torch.no_grad():
    for param in [layer.w, layer.b]:
      param.sub_(lr * param.grad)
    layer.zero_grad()


100%|██████████| 500/500 [00:00<00:00, 3364.92it/s]


 Loss for epoch: 50 is 2.3878281116485596

 Loss for epoch: 100 is 2.3562254905700684

 Loss for epoch: 150 is 2.3551907539367676

 Loss for epoch: 200 is 2.3551549911499023

 Loss for epoch: 250 is 2.355154037475586

 Loss for epoch: 300 is 2.3551533222198486

 Loss for epoch: 350 is 2.355153799057007

 Loss for epoch: 400 is 2.3551530838012695

 Loss for epoch: 450 is 2.3551530838012695

 Loss for epoch: 500 is 2.3551530838012695


In [119]:
layer.w, layer.b

(tensor([[4.6118],
         [5.3032],
         [1.0158],
         [1.2047]]),
 tensor([1.6510]))

# Code Refactoring

In [190]:
class MyModule:
  def __init__(self):
    self.parameters = []

  def get_parameters(self):
    return self.parameters

  def __call__(self, *args):
    self.args = args
    self.out = self.forward(*args)
    return self.out

  def _backward(self):
    raise Exception("Not Implemented")

  def backward(self):
    return self._backward(self.out, *self.args)

  def forward(self):
    raise Exception('Not Implemented')

In [191]:
class MSELoss(MyModule):
    def forward(self, targ, inp):
        return torch.mean((targ - inp.squeeze())**2)

    def _backward(self, out, targ, inp):
        grads = (-2/inp.shape[0])*(targ - inp.squeeze())
        inp.grad = grads.unsqueeze(-1)

class Relu(MyModule):
    def __init__(self):
        super().__init__()

    def forward(self, x):
        return torch.clamp(x, min = 0.)

    def _backward(self, out, inp):
        inp.grad = (inp > 0.0).float() * out.grad

class Linear(MyModule):
    def __init__(self, nin, nout):
        super().__init__()
        self.w = torch.randn(nin, nout)
        self.b = torch.rand(nout)
        self.parameters = [self.w, self.b]

    def get_parameters(self):
        return self.parameters

    def zero_grad(self):
        self.w.grad = torch.zeros_like(self.w)
        self.b.grad = torch.zeros_like(self.b)

    def forward(self, x):
        return torch.matmul(x, self.w) + self.b

    def _backward(self, out, inp):
        inp.grad = out.grad @ self.w.t()
        self.w.grad = inp.t() @ out.grad
        self.b.grad = out.grad.sum(0)


In [195]:
class Model(MyModule):
    def __init__(self, layers, loss_fn):
        self.layers = layers
        self.loss_fn = loss_fn

    def get_parameters(self):
        params = []
        for l in self.layers:
            params.extend(l.get_parameters())
        return params

    def forward(self, x):
        for l in self.layers:
            x = l(x)
        return x

    def _backward(self, *args):
        self.loss_fn.backward()

        for l in reversed(self.layers):
            l.backward()


In [196]:
Relu().get_parameters()

[]

In [209]:
loss_fn = MSELoss()
model = Model([Linear(4, 2), Relu(), Linear(2, 1)], loss_fn)
lr = 0.01
for i in tqdm(range(2000)):
  yhat = model(x)
  loss = model.loss_fn(y, yhat)
  model.backward()
  if (i + 1)%50 == 0:
    print(f"\n Loss for epoch: {i+1} is {loss.item()}")
  with torch.no_grad():
    for param in model.get_parameters():
      param.sub_(lr * param.grad)
    model.layers[0].zero_grad()
    model.layers[2].zero_grad()


 38%|███▊      | 750/2000 [00:00<00:00, 3575.61it/s]


 Loss for epoch: 50 is 19.24431037902832

 Loss for epoch: 100 is 10.260042190551758

 Loss for epoch: 150 is 6.667061805725098

 Loss for epoch: 200 is 4.986515522003174

 Loss for epoch: 250 is 3.5259690284729004

 Loss for epoch: 300 is 2.740647315979004

 Loss for epoch: 350 is 2.2755513191223145

 Loss for epoch: 400 is 1.9998388290405273

 Loss for epoch: 450 is 1.8372268676757812

 Loss for epoch: 500 is 1.741594910621643

 Loss for epoch: 550 is 0.9468549489974976

 Loss for epoch: 600 is 0.659949779510498

 Loss for epoch: 650 is 0.4763644337654114

 Loss for epoch: 700 is 0.34873297810554504

 Loss for epoch: 750 is 0.25921717286109924


 73%|███████▎  | 1464/2000 [00:00<00:00, 3496.17it/s]


 Loss for epoch: 800 is 0.1963081657886505

 Loss for epoch: 850 is 0.1520116925239563

 Loss for epoch: 900 is 0.12074178457260132

 Loss for epoch: 950 is 0.09859783202409744

 Loss for epoch: 1000 is 0.08285629004240036

 Loss for epoch: 1050 is 0.07161638885736465

 Loss for epoch: 1100 is 0.06354961544275284

 Loss for epoch: 1150 is 0.05134259909391403

 Loss for epoch: 1200 is 0.042517755180597305

 Loss for epoch: 1250 is 0.0352233462035656

 Loss for epoch: 1300 is 0.029184062033891678

 Loss for epoch: 1350 is 0.024182889610528946

 Loss for epoch: 1400 is 0.020039930939674377

 Loss for epoch: 1450 is 0.016607755795121193

 Loss for epoch: 1500 is 0.013763749971985817


100%|██████████| 2000/2000 [00:00<00:00, 3416.31it/s]


 Loss for epoch: 1550 is 0.011407050304114819

 Loss for epoch: 1600 is 0.009454063139855862

 Loss for epoch: 1650 is 0.00783574115484953

 Loss for epoch: 1700 is 0.006494194269180298

 Loss for epoch: 1750 is 0.005382300820201635

 Loss for epoch: 1800 is 0.004460818134248257

 Loss for epoch: 1850 is 0.0036970253568142653

 Loss for epoch: 1900 is 0.003063978161662817

 Loss for epoch: 1950 is 0.0025393334217369556

 Loss for epoch: 2000 is 0.0021044788882136345


In [210]:
model.layers[0].parameters, model.layers[2].parameters

([tensor([[-0.0710, -1.1680],
          [-1.9109, -1.4432],
          [ 0.8374, -0.5813],
          [ 1.5237, -0.3553]]),
  tensor([0.2737, 2.8973])],
 [tensor([[-0.0518],
          [-3.4404]]),
  tensor([11.9994])])